# Merge Data

In [107]:
import pandas as pd

In [108]:
data_path = '../data'
summary_path = f'{data_path}/MovieSummaries'

movie_summaries = pd.read_csv(f'{summary_path}/plot_summaries.csv')
cmu_df = pd.read_csv(f'{summary_path}/movie_metadata_wikidata.csv')
tmdb_df = pd.read_csv(f'{data_path}/TMDB_movie_dataset_v12.csv') 

In [109]:
cmu_df = pd.merge(cmu_df, movie_summaries, on='Wikipedia_movie_ID', how='left')

In [110]:
cmu_df["cmu_year"] = cmu_df["movie_year"].apply(lambda x: str(x).split('-')[0])
cmu_df["cmu_year"] = cmu_df["cmu_year"].apply(pd.to_numeric, errors='coerce')

tmdb_df['tmdb_year'] = tmdb_df['release_date'].apply(lambda x: str(x).split('-')[0])
tmdb_df['tmdb_year'] = tmdb_df['tmdb_year'].apply(pd.to_numeric, errors='coerce')

In [111]:
tmdb_df_out_nan = tmdb_df[tmdb_df['wikidata_id'].notnull()]
tmdb_only_cmu_df = tmdb_df_out_nan[tmdb_df_out_nan['wikidata_id'].isin(cmu_df['wikidata_id'])]

merged_df = pd.merge(cmu_df, tmdb_only_cmu_df, on='wikidata_id', how='inner')

In [112]:
# merge missing cmu data with tmdb data on title and year
missing_cmu_df = cmu_df[~cmu_df['wikidata_id'].isin(merged_df['wikidata_id'])]

merged_missing_df = pd.merge(missing_cmu_df, tmdb_df, left_on=["movie_name", "cmu_year"], right_on=["title", "tmdb_year"], how='left')
merged_missing_df = merged_missing_df[~merged_missing_df["cmu_year"].isna()]

merged_df = pd.concat([merged_df, merged_missing_df])

In [113]:
def fill_missing_values(origin_df, target_col, source_col):
    filter_mask = origin_df[target_col].isna()
    origin_df.loc[filter_mask,target_col] = origin_df[source_col][filter_mask]
    origin_df.drop(columns=[source_col], inplace=True)

In [114]:
target_fill_col = ['cmu_year', 'movie_revenue', 'movie_runtime', 'movie_countries', 'movie_genres', 'movie_summary']
source_fill_col = ['tmdb_year', 'revenue', 'runtime', 'production_countries', 'genres', 'overview']

for target, source in zip(target_fill_col, source_fill_col):
    fill_missing_values(merged_df, target, source)

In [115]:
merged_df.drop(columns=['spoken_languages', 'production_companies', 'movie_languages', 'status', 'backdrop_path', 'homepage', 'original_title', 'poster_path', 'tagline', 'release_date', 'movie_year', 'title'], inplace=True)

In [116]:
merged_df.drop(columns=['wikidata_id_y'], inplace=True)
merged_df.rename(columns={'wikidata_id_x': 'wikidata_id'}, inplace=True)

merged_df.rename(columns={'id': 'tmdb_id'}, inplace=True)

In [117]:
merged_df.loc[merged_df['vote_average'] == 0 , 'vote_average']  = None
merged_df.loc[merged_df['vote_count'] == 0 , 'vote_count']  = None
merged_df.loc[merged_df['budget'] == 0 , 'budget']  = None
merged_df.loc[merged_df['popularity'] == 0 , 'popularity']  = None
merged_df.loc[merged_df['movie_revenue'] == 0, 'movie_revenue'] = None
merged_df.loc[merged_df['movie_runtime'] == 0, 'movie_runtime'] = None

In [118]:
merged_df

,Wikipedia_movie_ID,Freebase_movie_ID,movie_name,movie_revenue,movie_runtime,movie_countries,movie_genres,wikidata_id,movie_summary,cmu_year,tmdb_id,vote_average,vote_count,adult,budget,imdb_id,original_language,popularity,keywords,wikidata_id
0,975900,/m/03vyhn,Ghosts of Mars,14010832.0,98.0,United States of America,"Science Fiction, Space western, Horror, Supern...",Q261700,"Set in the second half of the 22nd century, th...",2001.0,10016.0,5.127,977.0,False,28000000.0,tt0228333,en,14.189,"future, planet mars, anti hero, possession, ho...",NaN
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,NaN,95.0,United States of America,"Crime Drama, Mystery, Biographical film, Drama",Q16250726,Dramatization of the story behind the murder o...,2000.0,784579.0,NaN,NaN,False,NaN,tt0245916,en,0.600,"colorado, jonbenet",NaN
2,28463795,/m/0crgdbh,Brun bitter,NaN,83.0,Norway,"Crime Fiction, Drama",Q4978832,A stolen bicycle case ends with drunken detect...,1988.0,396302.0,NaN,NaN,False,NaN,tt0094806,no,0.898,"bicycle, private detective",NaN
3,9363483,/m/0285_cd,White Of The Eye,NaN,110.0,United Kingdom,"Psychological thriller, Thriller, Erotic thriller",Q7995657,A series of murders of rich young women throug...,1987.0,33592.0,5.742,64.0,False,NaN,tt0094320,en,8.297,"based on novel or book, gas station, psychopat...",NaN
4,261236,/m/01mrr1,A Woman in Flames,NaN,106.0,Germany,Drama,Q869644,"Eva, an upper class housewife, becomes frustra...",1983.0,11192.0,5.300,13.0,False,NaN,tt0083949,de,2.801,"jealousy, eroticism, gigolo, longing, dominatr...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10785,31322664,/m/0gjdl5d,Wohnkultur,NaN,NaN,German Democratic Republic,"Documentary, Short Film",NaN,NaN,1955.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q8029088
10786,23613845,/m/06zqzm0,The Trial,NaN,72.0,Ireland,Documentary,NaN,"In September 1998, at the end of the Balkan wa...",2009.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q7770136
10787,9998038,/m/06_y3t0,Morir por nada,NaN,NaN,Argentina,NaN,NaN,NaN,1974.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q12155803
10788,7762741,/m/0kvgqq,Business Versus Love,NaN,NaN,United States of America,"Silent film, Drama, Short Film",NaN,NaN,1914.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q5001814


In [119]:
merged_df.to_csv(f'{data_path}/enrich_movie_data.csv', index=False)